In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import os
from sklearn.linear_model import LinearRegression
from datetime import datetime
import math
from itertools import compress

In [2]:
from river_dl.preproc_utils import separate_trn_tst, read_multiple_obs, check_if_finite

In [3]:
os.chdir('/home/jovyan/river-dl')

In [4]:
out_file = os.path.join("DRB_output","prepped.npz")
obs_temper_file = os.path.join("DRB_data","obs_temp_subset")
pretrain_file = os.path.join("DRB_data","uncal_sntemp_input_output_subset")
test_start_date = '2010-09-30'
n_test_yr = 6

In [5]:
ds_pre = xr.open_zarr(pretrain_file)

In [6]:
obs = [ds_pre.sortby(["seg_id_nat","date"])]
obs.append(xr.open_zarr(obs_temper_file))
obs=xr.merge(obs,join="left")
obs=obs[["seg_tave_air","temp_c"]]
obs = obs.rename({"temp_c": "seg_tave_water"})

In [7]:
#split into testing and training
x_trn, x_tst = separate_trn_tst(obs, test_start_date, n_test_yr)

In [35]:
Date.data

array(['1980-10-01T00:00:00.000000000', '1980-10-02T00:00:00.000000000',
       '1980-10-03T00:00:00.000000000', ...,
       '2010-09-27T00:00:00.000000000', '2010-09-28T00:00:00.000000000',
       '2010-09-29T00:00:00.000000000'], dtype='datetime64[ns]')

In [37]:
(Date[0]-np.datetime64('1980-10-01'))

<xarray.DataArray 'date' ()>
array(0, dtype='timedelta64[ns]')
Coordinates:
    date     datetime64[ns] 1980-10-01

In [25]:
np.datetime64(Date[0],"D")

ValueError: Could not convert object to NumPy datetime

In [18]:
def annTempStats(thisData):
    #convert the date to decimal dates
    date_decimal = [float(x)/365 for x in ((thisData['date']-np.datetime64('1980-10-01'))/86400000000000.0)]
    
    air_amp=[]
    air_phi=[]
    water_amp_obs = []
    water_phi_obs = []
    nTemps = []
    
    #get the phase and amplitude for air and water temps for each segment
    for i in range(len(thisData['seg_id_nat'])):
        thisSeg = thisData['seg_id_nat'][i]
        tmean_air = thisData['seg_tave_air'][:,i]
        tmean_water = thisData['seg_tave_water'][i,:]
        x = [[math.sin(2*math.pi*j),math.cos(2*math.pi*j)] for j in date_decimal]
        model = LinearRegression().fit(x,tmean_air)
        amp = math.sqrt(model.coef_[0]**2+model.coef_[1]**2)
        phi = math.asin(model.coef_[1]/amp)
        air_amp.append(amp)
        air_phi.append(phi)
        if np.sum(np.isfinite(tmean_water).values)>365: #this requires at least 1 year of data, need to add other data requirements here
            model = LinearRegression().fit(list(compress(x, np.isfinite(tmean_water).values)),list(compress(tmean_water, np.isfinite(tmean_water).values)))
            amp = math.sqrt(model.coef_[0]**2+model.coef_[1]**2)
            phi = math.asin(model.coef_[1]/amp)
        else:
            amp=np.nan
            phi=np.nan
        water_amp_obs.append(amp)
        water_phi_obs.append(phi)
        nTemps.append(np.sum(np.isfinite(tmean_water).values))
    Ar_obs = [water_amp_obs[x]/air_amp[x] for x in range(len(water_amp_obs))]
    delPhi_obs = [(water_phi_obs[x]-air_phi[x])*365/(2*math.pi) for x in range(len(water_amp_obs))]
    
    tempDF = pd.DataFrame({'seg_id_nat':thisData['seg_id_nat'], 'air_amp':air_amp,'air_phi':air_phi,'water_amp_obs':water_amp_obs,'water_phi_obs':water_phi_obs,'Ar_obs':Ar_obs,'delPhi_obs':delPhi_obs})
    
    return tempDF
    

In [19]:
GW_trn = annTempStats(x_trn)
GW_tst = annTempStats(x_tst)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [84]:
preppedData = np.load(out_file)

In [81]:
data = {k:v for  k, v in preppedData.items() if not k.startswith("GW")}

In [82]:
data['GW_tst']=GW_tst
data['GW_trn']=GW_trn
data['GW_cols']=GW_trn.columns.values.astype('str')

In [83]:
np.savez_compressed(out_file, **data)

In [91]:
type(x_trn)

xarray.core.dataset.Dataset